#  Conexión a una Base de Datos MySQL desde Python:

**🚨NOTA IMPORTANTE: 🚨**

Al comenzar esta lección e ir ejecutando las diferentes celdas de Python, te recomendamos que tengas abierto tambien MySQL para ir viendo paso a paso como se van ejecutando las ordenes desde este notebook en MySQL.😊

En esta lección, aprenderemos cómo establecer una conexión con una base de datos MySQL existente desde un script de Python utilizando el constructor de la clase `connect()`.

El constructor `connect()` es esencial para crear una conexión con un servidor MySQL y devuelve un objeto de tipo `MySQLConnection`. A través de un ejemplo práctico, exploraremos cómo utilizarlo de manera efectiva.

🚨🚨 **NOTA IMPORTANTE** 🚨🚨 Probablemente no tengas instaladas estas librerías que vamos a utilizar, asi que si encuentras un error al ejecutar la celda de Python que esta debajo deberías ejecutar las instalaciones que se muestran a continuación, sigue estos pasos en tu terminal:

1. Ejecuta `pip install mysql-connector`.

<!-- 2. Luego, ejecuta `pip install mysql-connector-python`. -->

Asegúrate de que la terminal muestra "base" y que en Jupyter, en la parte superior derecha, se muestra "base (y la versión de Python)". Después de estos pasos, vuelve a ejecutar la celda.

## Descripción de los Argumentos de *`connect()`:*

Antes de comenzar, exploremos algunos de los argumentos clave que utilizaremos al conectar a una base de datos utilizando el constructor `connect()`.

- `user`: Este argumento se refiere al nombre de usuario que se utilizará para autenticarse en el servidor MySQL.

- `password`: Aquí debemos proporcionar la contraseña correspondiente al usuario para la autenticación en el servidor MySQL.

- `database`: Es el nombre de la base de datos a la que deseamos conectarnos.

- `host`: Este argumento se utiliza para especificar el nombre del servidor MySQL o la dirección IP a la que nos conectaremos. Por defecto, este valor es "localhost", que se refiere a la misma máquina en la que se ejecuta el código (127.0.0.1 es la dirección IP equivalente).

- `port`: El puerto TCP/IP del servidor MySQL. Debe ser un número entero y, por defecto, se establece en 3306.


Existen más argumentos que son opcionales o que se usarán cuando queramos configurar la conexión de una manera muy concreta. Podéis consultar la descripción de todos los parámetros [aquí](https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html). 

In [16]:
pip install mysql-connector

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode

In [3]:
# ahora creamos la conexión con los argumentos:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='tienda', 
                              auth_plugin ='mysql_native_password')

print(cnx)
cnx.close()


En ocasiones podemos sufrir errores de conexión, por lo que es interesante añadir manejo de excepciones al código anterior. Usando *errorcode* podemos crear casos para cada tipo de error posible:

In [28]:
# En este código estamos haciendo un try except. Si recordamos esto nos permitía hacer un manejo de los errores, para evitar que nuestro código se pare. Para eso lo que estamos haciendo es
## intenta hacer la conexión son la base de datos de tienda 
try:
  cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='tienda', auth_plugin ='mysql_native_password')
# en caso de que no lo consigas por que hay algún error entonces ...
except mysql.connector.Error as err:

  # si es un error con la contraseña devuelveme un mensaje de acceso denegado ya que tenemos problemas con la contraseña
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  
  # si el error no tiene que ver con la contraseña, puede ser porque la base de datos no exista, devuelveme un mensaje de que la base de datos no existe
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  
  # si no es por ninguno de los errores anteriores, printeame cual es el error que estoy teniendo en mi conexión
  else:
    print(err)
else:
  cnx.close()

## Desconexión de la base de datos usando MySQLConnector

Una vez hayamos terminado de realizar las consultas o trabajar de cualquier modo con la base de datos a la que nos hemos conectado con el connector, tendremos que desconectarnos usando el método *close()*:

In [41]:
# En este caso vamos a pasarle los atributos al conector usando un diccionario:

config = {
  'user': 'root',
  'password': 'AlumnaAdalab',
  'host': '127.0.0.1',
  'database': 'tienda',
  'auth_plugin': 'mysql_native_password',
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)

cnx.close()

En este fragmento de código, hemos establecido la conexión `cnx` con la base de datos "tienda". Con esta conexión, podemos llevar a cabo consultas sobre los datos almacenados en sus tablas (más adelante exploraremos cómo hacerlo). Cuando hayamos finalizado nuestras operaciones y deseemos desconectarnos, será necesario utilizar el método `close()` del objeto connector que hemos definido, en este caso llamado `cnx`. Es importante destacar que este método no requiere argumentos, lo que lo hace de uso sencillo.

# Realización de queries usando MySQLConnector:

Una vez que creamos la conexión con mysql, vamos a usar el método *cursor()* y *execute()* para poder realizar las queries.

El método cursor() se utiliza para crear un objeto de cursor, que es esencial para ejecutar consultas SQL en una conexión de base de datos. Una vez que tienes un objeto de cursor, puedes utilizar el método execute() para enviar una consulta SQL al servidor MySQL y ejecutarla.

In [31]:
# realizamos la conexión con la BBDD que queremos
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='tienda', auth_plugin ='mysql_native_password')

# iniciamos el cursor                               
mycursor = cnx.cursor()

# query de ejemplo:

query = ("""SELECT city, state FROM customers 
         WHERE customer_number BETWEEN 121 AND 124""")

# luego llamamos al método execute() del cursor, al que pasamos como argumento el string que contiene la consulta.
## De esta manera se ejecuta la consulta en la base de datos y si todo va correctamente, el resultado se almacenará en el cursor.
mycursor.execute(query)

En el *execute()* podemos pasar cualquier tipo de query: crear tablas, alterar tablas, insertar datos,... vamos a verlo con unos ejemplos.

Vamos a aprovechar para introducir el manejo de excepciones a la hora de trabajar con MySQL Connector/Python.

- Crear Bases de Datos:

In [32]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', auth_plugin ='mysql_native_password')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE DATABASE BD_pruebas")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

MySQLCursor: CREATE DATABASE BD_pruebas


**Ojo:** al realizar la conexión, no le hemos indicado ninguna bases de datos, ya que la estamos creando.

En este ejemplo, estamos estableciendo la conexión y creando el *cursor*, después intentamos crear la base de datos "BD_pruebas". Si no se puede crear, nos dará un mensaje con el error que ha ocurrido.

En este momento, notaras que en MySQL existe una nueva base de datos llamada BD_pruebas (puedes verla en MySQL junto a los demas SCHEMAS)

Si ejecutamos dos veces esta celda, nos dará un aviso diendo que la base de datos ya existe.

- Creación de tablas:

Ya tenemos una base de datos creada, pero seguramente sabes que no tiene nada, ni siquiera su unidad basica que son las tablas, asi que comenzaremos creando una llama "customers".

In [33]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas', auth_plugin ='mysql_native_password')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE TABLE customers (name VARCHAR(255), address VARCHAR(255))")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

MySQLCursor: CREATE TABLE customers (name VARCHAR(255..


Ahora sí hemos establecido la base de datos sobre la que estamos trabajando, por lo demás, este código es igual que el anterior, cambiando la query para crear una tabla en nuestra base de datos.

- Inserción de datos:

Ahora que hemos completado la creación de una base de datos, hemos definido una tabla con sus columnas y restricciones correspondientes, estamos listas para comenzar a agregar registros a esta tabla.

In [34]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas', auth_plugin ='mysql_native_password')


mycursor = cnx.cursor()
query = "INSERT INTO customers (name, address) VALUES (%s, %s)"
val = ("Ana", "Calle 21")
try: 
    mycursor.execute(query, val)
    cnx.commit()
    print(mycursor.rowcount, "registro insertado.")  #rowcount ???

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1 registro insertado.


Seguimos trabajando sobre BD_pruebas. Ahora hemos definido nuestra query para la inserción de datos en la que hemos incluido `%s` por cada uno de los valores que vamos a introducir. Después, definimos una variable en la que van los datos que queremos insertar en forma de tupla.

Esta vez, en el *execute()* hemos pasado dos argumentos: el primero con la query y el segundo con los valores.

**Uso de commit:** A la hora de trabajar con sentencias INSERT es necesario el uso de *commit()* para que los cambios se efectuen en la base de datos. De no llamar a ese método, las inserciones no se llevarán a cabo.

- Inserción de múltiples registros:

En el ejemplo anterior hemos introducido una única fila en nuestra base de datos. Si queremos insertar múltiples filas tenemos que usar *executemany()* y definir una variable con todos los datos que queremos insertar en forma de lista de tuplas.

In [35]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas', auth_plugin ='mysql_native_password')


mycursor = cnx.cursor()
sql = "INSERT INTO customers (name, address) VALUES (%s, %s)"
val = [
  ('Ana', 'Lowstreet 4'),
  ('Rocio', 'Apple st 652'),
  ('Juana', 'Mountain 21'),
  ('Pedro', 'Valley 345')
]

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

4 registro/s insertado/s.


- Eliminar registros de una tabla:

**Uso de rollback:** Si después de haber realizado una transacción con *execute()* nos diésemos cuenta de que los datos que hemos introducido son incorrectos en alguna manera, si aún no hemos ejecutado *commit()* podríamos dar marcha atrás y desestimar los cambios usando *rollback()*:

In [36]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas', auth_plugin ='mysql_native_password')


mycursor = cnx.cursor()
sql = "INSERT INTO customers (name, address) VALUES (%s, %s)"
val = ("Lucia", "Plaza 22")
try:
    mycursor.execute(sql, val)
    cnx.rollback()
    print(mycursor.rowcount, "registro no insertado.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

mycursor.execute("SELECT * FROM customers")  
results = mycursor.fetchall()
print(results)

1 registro no insertado.
[('Ana', 'Calle 21'), ('Ana', 'Lowstreet 4'), ('Rocio', 'Apple st 652'), ('Juana', 'Mountain 21'), ('Pedro', 'Valley 345')]


Si ya hemos ejecutado *commit()* y queremos eliminar un registro de una tabla existente, usaremos *execute()* con la sentencia "DELETE FROM" de la siguiente manera:

In [37]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas', auth_plugin ='mysql_native_password')


mycursor = cnx.cursor()
sql = "DELETE FROM customers WHERE address = 'Calle 21'"
try:
    mycursor.execute(sql)
    cnx.commit()
    print(mycursor.rowcount, "registro/s eliminado/s")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1 registro/s eliminado/s


- Actualizar una tabla:

También podemos actualizar registros de la misma forma que lo haríamos en SQL con la sentencia "UPDATE":

In [38]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas', auth_plugin ='mysql_native_password')


mycursor = cnx.cursor()
sql = "UPDATE customers SET address = 'Canyon 123' WHERE address = 'Valley 345'"
try:
    mycursor.execute(sql)
    cnx.commit()
    print(mycursor.rowcount, "registro/s modificado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1 registro/s modificado/s.


**NOTA:** siempre que hagamos cualquier modificación sobre los registros tenemos que usar *commit()* para que esos cambios se hagan efectivos.

Hemos llegado al final de esta lección sobre cómo conectar **MySQL** con **Python**. 🚀 Hoy hemos cubierto un tema clave como es la conexión a bases de datos desde Python

Con esta base sólida, ya tienes las herramientas necesarias para empezar a trabajar con bases de datos de manera más dinámica y eficiente, integrando Python en el proceso. 💻⚙️

Recuerda que la práctica es clave 🔑. Te animo a seguir experimentando con diferentes consultas, datasets y conexiones. ¡Cuanto más practiques, más dominarás esta poderosa combinación!

¡Hasta pronto y sigue con ese gran espíritu de aprendizaje! 💪
